In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import string
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
import pandas as pd

# Show full content in each cell
pd.set_option('display.max_colwidth', None)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Amazon_ML/student_resource/dataset/train.csv')
train.head()

,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n",https://m.media-amazon.com/images/I/51mo8htwTHL.jpg,4.89
1,198967,"Item Name: Salerno Cookies, The Original Butter Cookies, 8 Ounce (Pack of 4)\nBullet Point 1: Original Butter Cookies: Classic butter cookies made with real butter\nBullet Point 2: Variety Pack: Includes 4 boxes with 32 cookies total\nBullet Point 3: Occasion Perfect: Delicious cookies for birthdays, weddings, anniversaries\nBullet Point 4: Shareable Treats: Fun to give and enjoy with friends and family\nBullet Point 5: Salerno Brand: Trusted brand of delicious butter cookies since 1925\nValue: 32.0\nUnit: Ounce\n",https://m.media-amazon.com/images/I/71YtriIHAAL.jpg,13.12
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy Chicken with Rice, 1.9 Ounce (Pack of 6)\nBullet Point 1: Loaded with hearty long grain wild rice and vegetables\nBullet Point 2: Full of hearty goodness\nBullet Point 3: Single serve bowls\nBullet Point 4: Easy to prepare mix\nBullet Point 5: 0 grams trans fat\nValue: 11.4\nUnit: Ounce\n",https://m.media-amazon.com/images/I/51+PFEe-w-L.jpg,1.97
3,55858,"Item Name: Judee’s Blue Cheese Powder 11.25 oz - Gluten-Free and Nut-Free - Use in Seasonings and Salad Dressings - Great for Dips, Spreads and Sauces - Made in USA\nBullet Point 1: Add to your favorite appetizers, dips & spreads. Use to season popcorn or warmed pita chips.\nBullet Point 2: Sprinkle over french fries, fried chicken, mashed potatoes, roasted veggies, pasta, and more\nBullet Point 3: Made in a dedicated gluten-free facility and shipped in a standup, resealable pouch to ensure freshness\nBullet Point 4: Ingredients: Blue Cheese (Milk, Salt, Cultures, & Enzymes) and Disodium Phosphate\nBullet Point 5: Since 2009, Judee’s has been dedicated to providing fresh, allergy-conscious ingredients, great for your recipes and even better for your family\nProduct Description: Judees Powdered Blue Cheese cheddar cheese powder is an alternative to mozzarella cheese shredded or american cheese slices deli. Make your own alfredo sauce with heavy cream and black buffalo dip with this powder. It adds extra flavor to salad dressing like ranch dressing and great on pizza dough or cauliflower pasta. Add to macaroni and cheese or popcorn seasoning for more aroma and cheesy feel. Combine with mustard and other ingredients to create your own dressing for buffalo chicken or buffalo wings.\nValue: 11.25\nUnit: Ounce\n",https://m.media-amazon.com/images/I/41mu0HAToDL.jpg,30.34
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Ounce - 12 per case.\nBullet Point: kedem Sherry Cooking Wine, 12.7 Ounce - 12 per case.\nValue: 12.0\nUnit: Count\n",https://m.media-amazon.com/images/I/41sA037+QvL.jpg,66.49


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Amazon_ML/student_resource/dataset/test.csv')
test.head()

,sample_id,catalog_content,image_link
0,100179,"Item Name: Rani 14-Spice Eshamaya's Mango Chutney (Indian Preserve) 10.5oz (300g) Glass Jar, Ready to eat, Vegan ~ Gluten Free, All Natural, NON-GMO\nBullet Point 1: You'll LOVE our 14-Spice Eshamaya's Mango Chutney Chutney by Rani Brand--Here's Why:\nBullet Point 2: 🥭All Natural, Non-GMO, No Preservatives, Gluten Free PREMIUM Gourmet Food Grade Chutney.\nBullet Point 3: 🥭Authentic Family Rani Recipe. Rani is a USA based company selling Indian Foods for over 40 years, buy with confidence!\nBullet Point 4: 🥭Great Spicy Taste, Awesome with Wine & Cheese! Packaged in a re-closeable glass jar\nBullet Point 5: 🥭Net Wt. 10.5oz - Authentic Product of India\nProduct Description: Mango chutney is made from diced green mangoes cooked with sugar. Spices are added to distinguish the different types of chutneys. Hot mango chutney is spiced with chilli powder which give the chutney heat and a deep amber color. Sweet Mango chutney allows the pallet to appreciate the flavor of the mango, adding a hint of vinegar for sourness, while major grey chutney mixes raisins, ginger powder, chilli powder and an array of spices to give complexity to the chutney. Try Rani Chutney's with pizza, salads, burgers, sandwiches, cheeses and anything else you can imagine!\nValue: 10.5\nUnit: Ounce\n",https://m.media-amazon.com/images/I/71hoAn78AWL.jpg
1,245611,"Item Name: Natural MILK TEA Flavoring extract by HALO PANTRY (2oz bottle) | Perfect for customizing ANY baking, candy, dessert or drink recipes | Dairy Free, Dye-Free, Gluten Free, Zero Sugars, Zero Calories, Gourmet (Pack of 1)\nBullet Point 1: Authentic Tasting, Asian-Inspired Natural flavorings made exclusively for use in any baking, cooking, dessert, candy, or drink recipes.\nBullet Point 2: NATURAL : Gluten-Free, Dairy-Free, Zero Sugars, Zero Calories, Soy-Free, Sodium Free, GMO-Free, PG-Free, Oil-Free\nBullet Point 3: No Colors, No Sweeteners, No Artificial Ingredients, No Preservatives\nBullet Point 4: All ingredients & packaging sourced and made within the USA\nBullet Point 5: Tastes like Brown Sugar bubble milk teas\nBullet Point 6: new 2oz Professional Series bottle for frequent bakers and cooks.\nProduct Description: Check our popular Milk Tea flavoring extract in a New larger bottle for use in any food or drink recipes. Perfect substitute for your average vanilla extract if you want to amplify the taste of your recipes. *HALO PANTRY is a premium line of Asian-inspired flavorings made exclusively for use in any baked goods, cooking, dessert, candy, or drink recipes. Our product line is very aromatic & flavorful to help enhance your recipes. The mission of HALO PANTRY brand is to make the DIY, at-home cooking/baking experience simple, fun, and to also encourage others to explore asian-inspired flavors. *Unlike other food flavoring products, Halo Pantry only adds flavors & does not alter or negatively impact the taste of your original recipe because it doesn't contain any sweeteners or other additives. With our product, you have complete control of your own recipes.\nValue: 2.0\nUnit: Fl Oz\n",https://m.media-amazon.com/images/I/61ex8NHCIjL.jpg
2,146263,"Item Name: Honey Filled Hard Candy - Bulk Pack 2 Pounds - Individually Wrapped\nBullet Point 1: Honey Filled Hard Candy; 2-pound bulk pack; approximately 180 pieces per order\nBullet Point 2: Treat yourself to the soothing sweetness of candy that captures the essence of natural honey\nBullet Point 3: Each piece is expertly crafted with a hard candy shell that gives way to a luscious, liquid honey center\nBullet Point 4: Suitable for soothing a sore throat, sweetening a cup of tea, or simply enjoying as a sweet snack\nBullet Point 5: Whether at home, in the office, or on the move, this candy will satisfy your sweet tooth and offer a moment of indulgence\nProduct Description: Honey Filled Hard Candy - Bulk Pack 2 Pounds - Individually Wrapped\nValue: 32.0\nUnit: Ounce\n",https://m.media-amazon.com/imag

In [ ]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

Train shape: (75000, 4)
Test shape: (75000, 3)


,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n",https://m.media-amazon.com/images/I/51mo8htwTHL.jpg,4.89
1,198967,"Item Name: Salerno Cookies, The Original Butter Cookies, 8 Ounce (Pack of 4)\nBullet Point 1: Original Butter Cookies: Classic butter cookies made with real butter\nBullet Point 2: Variety Pack: Includes 4 boxes with 32 cookies total\nBullet Point 3: Occasion Perfect: Delicious cookies for birthdays, weddings, anniversaries\nBullet Point 4: Shareable Treats: Fun to give and enjoy with friends and family\nBullet Point 5: Salerno Brand: Trusted brand of delicious butter cookies since 1925\nValue: 32.0\nUnit: Ounce\n",https://m.media-amazon.com/images/I/71YtriIHAAL.jpg,13.12
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy Chicken with Rice, 1.9 Ounce (Pack of 6)\nBullet Point 1: Loaded with hearty long grain wild rice and vegetables\nBullet Point 2: Full of hearty goodness\nBullet Point 3: Single serve bowls\nBullet Point 4: Easy to prepare mix\nBullet Point 5: 0 grams trans fat\nValue: 11.4\nUnit: Ounce\n",https://m.media-amazon.com/images/I/51+PFEe-w-L.jpg,1.97
3,55858,"Item Name: Judee’s Blue Cheese Powder 11.25 oz - Gluten-Free and Nut-Free - Use in Seasonings and Salad Dressings - Great for Dips, Spreads and Sauces - Made in USA\nBullet Point 1: Add to your favorite appetizers, dips & spreads. Use to season popcorn or warmed pita chips.\nBullet Point 2: Sprinkle over french fries, fried chicken, mashed potatoes, roasted veggies, pasta, and more\nBullet Point 3: Made in a dedicated gluten-free facility and shipped in a standup, resealable pouch to ensure freshness\nBullet Point 4: Ingredients: Blue Cheese (Milk, Salt, Cultures, & Enzymes) and Disodium Phosphate\nBullet Point 5: Since 2009, Judee’s has been dedicated to providing fresh, allergy-conscious ingredients, great for your recipes and even better for your family\nProduct Description: Judees Powdered Blue Cheese cheddar cheese powder is an alternative to mozzarella cheese shredded or american cheese slices deli. Make your own alfredo sauce with heavy cream and black buffalo dip with this powder. It adds extra flavor to salad dressing like ranch dressing and great on pizza dough or cauliflower pasta. Add to macaroni and cheese or popcorn seasoning for more aroma and cheesy feel. Combine with mustard and other ingredients to create your own dressing for buffalo chicken or buffalo wings.\nValue: 11.25\nUnit: Ounce\n",https://m.media-amazon.com/images/I/41mu0HAToDL.jpg,30.34
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Ounce - 12 per case.\nBullet Point: kedem Sherry Cooking Wine, 12.7 Ounce - 12 per case.\nValue: 12.0\nUnit: Count\n",https://m.media-amazon.com/images/I/41sA037+QvL.jpg,66.49


In [ ]:
train.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0
price,0


In [ ]:
# Function to show details
def dataset_info(name, df):
    print(f"📊 Dataset: {name}")
    print("="*50)
    print("Shape (rows, columns):", df.shape)
    print("\nColumn Names:", df.columns.tolist())
    print("\nData Types:\n", df.dtypes)
    print("\nMissing Values:\n", df.isnull().sum())
    print("\nFirst 5 Rows:\n", df.head())
    print("\n" + "="*50 + "\n")

In [ ]:
# Show details for both
dataset_info("Trainung Data Information", train)


📊 Dataset: Trainung Data Information
Shape (rows, columns): (75000, 4)

Column Names: ['sample_id', 'catalog_content', 'image_link', 'price']

Data Types:
 sample_id            int64
catalog_content     object
image_link          object
price              float64
dtype: object

Missing Values:
 sample_id          0
catalog_content    0
image_link         0
price              0
dtype: int64

First 5 Rows:
    sample_id  \
0      33127   
1     198967   
2     261251   
3      55858   
4     292686   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
dataset_info("Testing Data Information", test)

📊 Dataset: Testing Data Information
Shape (rows, columns): (75000, 3)

Column Names: ['sample_id', 'catalog_content', 'image_link']

Data Types:
 sample_id           int64
catalog_content    object
image_link         object
dtype: object

Missing Values:
 sample_id          0
catalog_content    0
image_link         0
dtype: int64

First 5 Rows:
    sample_id  \
0     100179   
1     245611   
2     146263   
3      95658   
4      36806   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
# ===============================
# 1️⃣ Text Cleaning Function
# ===============================
def clean_text(text):

    if not isinstance(text, str):
        return ""

    # Lowercase and trim
    text = text.lower().strip()

    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)

    # Remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)

    # Remove emojis and non-standard unicode symbols
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002700-\U000027BF"  # dingbats
        u"\U0001F900-\U0001F9FF"  # supplemental symbols
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Keep letters, numbers, and meaningful symbols: + - x % . /
    text = re.sub(r"[^a-z0-9\s\+\-x%./]", " ", text)

    # Collapse multiple spaces/newlines into single space
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
# ===============================
# 2️⃣ Apply Preprocessing
# ===============================
train['clean_text'] = train['catalog_content'].fillna("").apply(clean_text)
test['clean_text'] = test['catalog_content'].fillna("").apply(clean_text)

In [ ]:
# ===============================
# 3️⃣ Tokenization & Padding
# ===============================
MAX_NUM_WORDS = 20000
MAX_SEQ_LEN = 100  # max words per product description

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(train['clean_text'])

X_train_seq = tokenizer.texts_to_sequences(train['clean_text'])
X_test_seq = tokenizer.texts_to_sequences(test['clean_text'])

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

In [ ]:
# ===============================
# 4️⃣ Train/Validation Split
# ===============================
y_train = train['price'].values
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_pad, y_train, test_size=0.2, random_state=42
)

In [ ]:
# ===============================
# 5️⃣ Build LSTM Model
# ===============================
EMBEDDING_DIM = 100

model = Sequential([
    Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQ_LEN),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')  # regression output
])

model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ===============================
# 6️⃣ Train the Model
# ===============================
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=128
)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 190s 396ms/step - loss: 17.2182 - val_loss: 16.2276
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 195s 382ms/step - loss: 15.3250 - val_loss: 14.2869
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 181s 386ms/step - loss: 13.5109 - val_loss: 13.3824
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 179s 381ms/step - loss: 11.9495 - val_loss: 12.4446
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 180s 385ms/step - loss: 10.6854 - val_loss: 12.0903


In [ ]:

# ===============================
# 7️⃣ Validate
# ===============================
y_val_pred = model.predict(X_val).flatten()
mae = mean_absolute_error(y_val, y_val_pred)
print(f"Validation MAE: {mae:.2f}")

# ===============================
# 8️⃣ Predict on Test Set
# ===============================
test_preds = model.predict(X_test_pad).flatten()
test_preds = np.maximum(test_preds, 0)  # Ensure positive prices

469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step
Validation MAE: 12.09
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 115s 49ms/step


In [ ]:
# Performance Metrics

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_pred) + np.abs(y_true)) / 2
    smape_val = np.mean(numerator / denominator) * 100
    return smape_val

# Example
y_true = np.array([100, 200, 300])
y_pred = np.array([120, 190, 310])

print("SMAPE:", smape(y_true, y_pred))

SMAPE: 8.862903944871157


In [ ]:

# ===============================
# 10 Prepare Submission
# ===============================
submission = pd.DataFrame({
    "sample_id": test['sample_id'],
    "price": test_preds
})

submission.to_csv("test_out.csv", index=False)
print("Submission file 'test_out.csv' created successfully!")

Submission file 'test_out.csv' created successfully!


In [ ]:
# -*- coding: utf-8 -*-
"""
Fast LSTM Predictor for Amazon Product Prices
"""

import os
import re
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
MAX_SEQ_LEN = 120  # must match training

# -------------------------
# Text Cleaning
# -------------------------
def clean_text(texts):
    """
    Clean a list of texts
    """
    cleaned = []
    for text in texts:
        if not isinstance(text, str):
            text = ""
        text = text.lower()
        text = re.sub(r"<.*?>", " ", text)
        text = re.sub(r"https?://\S+|www\.\S+", " ", text)
        text = re.sub(r"[^\w\s]", " ", text)
        text = re.sub(r"\d+oz", " ounce ", text)
        text = re.sub(r"\d+fl\s*oz", " ounce ", text)
        text = re.sub(r"\d+lb", " pound ", text)
        text = re.sub(r"\s+", " ", text)
        text = ' '.join([w for w in text.split() if w not in stop_words])
        cleaned.append(text.strip())
    return cleaned

# -------------------------
# Load Model + Tokenizer + Unit Encoder
# -------------------------
lstm_model = load_model('models/lstm_model.h5', compile=False)
with open('models/tokenizer_lstm.pkl', 'rb') as f:
    lstm_tokenizer = pickle.load(f)
with open('models/unit_encoder.pkl', 'rb') as f:
    unit_encoder = pickle.load(f)

# -------------------------
# SMAPE
# -------------------------
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    denominator[denominator == 0] = 1
    return np.mean(np.abs(y_true - y_pred) / denominator) * 100

# -------------------------
# Vectorized Feature Extraction
# -------------------------
def extract_numeric_features(df):
    # Value
    df['Value_feat'] = df['catalog_content'].str.extract(r"Value[:\s]*([\d\.]+)")[0]
    df['Value_feat'] = pd.to_numeric(df['Value_feat'], errors='coerce').fillna(1.0)

    # Unit
    df['Unit'] = df['catalog_content'].str.extract(r"Unit[:\s]*([a-zA-Z]+)")[0].fillna("NA")
    # Map unseen units to 0
    df['Unit_feat'] = df['Unit'].apply(lambda x: unit_encoder.transform([x])[0] if x in unit_encoder.classes_ else 0)

    return df[['Value_feat', 'Unit_feat']].values

# -------------------------
# Main Script
# -------------------------
if __name__ == "__main__":
    DATASET_FOLDER = '/content/drive/MyDrive/Amazon_ML/student_resource/dataset'

    test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

    # 1. Clean text
    cleaned_texts = clean_text(test['catalog_content'].tolist())

    # 2. Tokenize and pad sequences
    sequences = lstm_tokenizer.texts_to_sequences(cleaned_texts)
    padded_texts = pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

    # 3. Numeric features
    numeric_features = extract_numeric_features(test)

    # 4. Predict in batch
    batch_preds = lstm_model.predict([padded_texts, numeric_features], batch_size=512, verbose=1)
    test['price'] = np.clip(batch_preds.flatten(), 0.01, None)  # ensure non-negative

    # 5. Save CSV
    output_path = os.path.join(DATASET_FOLDER, 'test_out_fast.csv')
    test[['sample_id', 'price']].to_csv(output_path, index=False, float_format='%.4f')

    print(f"✅ Predictions saved to {output_path}")
    print(test[['sample_id', 'price']].head())


In [ ]:
# -*- coding: utf-8 -*-
"""
Fast LSTM Predictor for Amazon Product Prices
"""

import os
import re
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
MAX_SEQ_LEN = 120  # must match training

# -------------------------
# Text Cleaning
# -------------------------
def clean_text(texts):
    """
    Clean a list of texts
    """
    cleaned = []
    for text in texts:
        if not isinstance(text, str):
            text = ""
        text = text.lower()
        text = re.sub(r"<.*?>", " ", text)
        text = re.sub(r"https?://\S+|www\.\S+", " ", text)
        text = re.sub(r"[^\w\s]", " ", text)
        text = re.sub(r"\d+oz", " ounce ", text)
        text = re.sub(r"\d+fl\s*oz", " ounce ", text)
        text = re.sub(r"\d+lb", " pound ", text)
        text = re.sub(r"\s+", " ", text)
        text = ' '.join([w for w in text.split() if w not in stop_words])
        cleaned.append(text.strip())
    return cleaned

# -------------------------
# Load Model + Tokenizer + Unit Encoder
# -------------------------
lstm_model = load_model('models/lstm_model.h5', compile=False)
with open('models/tokenizer_lstm.pkl', 'rb') as f:
    lstm_tokenizer = pickle.load(f)
with open('models/unit_encoder.pkl', 'rb') as f:
    unit_encoder = pickle.load(f)

# -------------------------
# SMAPE
# -------------------------
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    denominator[denominator == 0] = 1
    return np.mean(np.abs(y_true - y_pred) / denominator) * 100

# -------------------------
# Vectorized Feature Extraction
# -------------------------
def extract_numeric_features(df):
    # Value
    df['Value_feat'] = df['catalog_content'].str.extract(r"Value[:\s]*([\d\.]+)")[0]
    df['Value_feat'] = pd.to_numeric(df['Value_feat'], errors='coerce').fillna(1.0)

    # Unit
    df['Unit'] = df['catalog_content'].str.extract(r"Unit[:\s]*([a-zA-Z]+)")[0].fillna("NA")
    # Map unseen units to 0
    df['Unit_feat'] = df['Unit'].apply(lambda x: unit_encoder.transform([x])[0] if x in unit_encoder.classes_ else 0)

    return df[['Value_feat', 'Unit_feat']].values

# -------------------------
# Main Script
# -------------------------
if __name__ == "__main__":
    DATASET_FOLDER = '/content/drive/MyDrive/Amazon_ML/student_resource/dataset'

    test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

    # 1. Clean text
    cleaned_texts = clean_text(test['catalog_content'].tolist())

    # 2. Tokenize and pad sequences
    sequences = lstm_tokenizer.texts_to_sequences(cleaned_texts)
    padded_texts = pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

    # 3. Numeric features
    numeric_features = extract_numeric_features(test)

    # 4. Predict in batch
    batch_preds = lstm_model.predict([padded_texts, numeric_features], batch_size=512, verbose=1)
    test['price'] = np.clip(batch_preds.flatten(), 0.01, None)  # ensure non-negative

    # 5. Save CSV
    output_path = os.path.join(DATASET_FOLDER, 'test_out_fast.csv')
    test[['sample_id', 'price']].to_csv(output_path, index=False, float_format='%.4f')

    print(f"✅ Predictions saved to {output_path}")
    print(test[['sample_id', 'price']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


147/147 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step
✅ Predictions saved to /content/drive/MyDrive/Amazon_ML/student_resource/dataset/test_out_fast.csv
   sample_id      price
0     100179  12.791854
1     245611  11.608626
2     146263  23.875540
3      95658   2.951736
4      36806  36.983425
